In [1]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import SimpleITK as sitk
import numpy as np
%matplotlib inline

In [2]:
def swc_to_dataframe( file_name ) :
    col_names = ['id', 'type', 'x', 'y', 'z', 'radius', 'parent', 'seg_id', 'level', 'mode', 'timestamp', 'TFresindex']
    df = pd.read_csv( file_name, sep=r"\s+", comment = "#", names = col_names, index_col = 'id')
    return df

In [3]:
def filter_swc( df, type ) :
    dd = df[df['type']==type]
    return dd

In [4]:
model_directory = "/allen/programs/celltypes/production/0378/informatics/model/P56"
image_directory = os.path.join( model_directory, "atlasVolume")

model_directory = "/home/lydian/informatics_lydian/Morphology/WholeToSliceView"
swc_directory = os.path.join( model_directory, "mouselight", "isocortex", "left_hemisphere")
output_directory = os.path.join( model_directory, "mouselight", "isocortex", "left_hemisphere_thumbnail")

In [5]:
#
# component types
#
types = {}
types['soma'] = {'type_id': 1, 'color': 'black'}
types['axon'] = {'type_id': 2, 'color': 'cyan'}
types['basal dendrite'] = {'type_id': 3, 'color': 'orangered'}
types['apical dendrite'] = {'type_id': 4, 'color': 'maroon'}

#
# views for swc
#
views = {}
views['coronal'] = {'order': 0, 'x': 'z', 'y': 'y'} #x
views['sagittal'] = {'order': 1, 'x': 'x', 'y' : 'y'} #y
views['horizontal'] = {'order': 2, 'x': 'x', 'y' : 'z'} #z

#
# image slices
#
slices = {}
slices['coronal'] = {'swc': 'x', 'image' : 'x', 'transpose' : True }
slices['sagittal'] = {'swc': 'z', 'image' : 'z', 'transpose' : False }
slices['horizontal'] = {'swc' : 'y', 'image' : 'y', 'transpose' : False }

#
# scale from image pixel index to SWC coordinate
#
scales = {}
scales['z'] = 10
scales['x'] = 10
scales['y'] = 10

shifts = {}
shifts['z'] = 0
shifts['x'] = 0
shifts['y'] = 0

In [6]:
#
# Create morphology thumbnail for each SWC file in the swc directory
#
#

image_file = os.path.join(image_directory, "average_template_10.nrrd" )
image = sitk.ReadImage( image_file )
print(image.GetSize())

for swc_file in glob.glob(os.path.join(swc_directory, "*.swc")):
    
    bb = os.path.basename( swc_file )
    
    # find the associated brain file
    bname = bb.split('_')[0]
    
    # set the output thumbnail png file name
    cname = bb.split('.')[0]
    thumnail_file = os.path.join( output_directory, "%s.png" % cname )
    
    print((cname,bname))
    
    #
    # read swc components into dataframe
    #
    df = swc_to_dataframe( swc_file )


    #
    # scale swc coordinates to match image volume
    #
    for sk, sv in shifts.items() :
        df[sk] = df[sk] + sv

    for sk, sv in scales.items() :
        df[sk] = df[sk] / sv
        

    #
    # get background image from soma location
    #

    filtered = filter_swc(df,types['soma']['type_id'])
    
    if len(filtered) == 0 :
        print("no soma found")
        continue
        
    soma = filtered.iloc[0]
 
    #
    #  plot swc components onto of image at soma location
    #
    plt.rcParams["figure.figsize"] = (30,8)

    fig, axs = plt.subplots(1,3)
    fig.suptitle(os.path.basename(swc_file))

    for vk, vv in views.items() :

        idx = int(round(soma[slices[vk]['swc']],0))

        if slices[vk]['image'] == 'x' :
            bkg = sitk.GetArrayViewFromImage( image )[:,:,idx]
        elif slices[vk]['image'] == 'y' :
            bkg = sitk.GetArrayViewFromImage( image )[:,idx,:]
        elif slices[vk]['image'] == 'z' :
            bkg = sitk.GetArrayViewFromImage( image )[idx,:,:]

        if slices[vk]['transpose'] :
            bkg = bkg.transpose()

        axs[vv['order']].imshow( bkg, cmap='gray')

        for tk,tv in types.items() :

            filtered = filter_swc(df, tv['type_id'])

            x = filtered[vv['x']].values
            y = filtered[vv['y']].values

            axs[vv['order']].scatter( x, y, s=0.01, c=tv['color'])

        axs[vv['order']].set_aspect('equal')

    
    plt.savefig(thumnail_file, dpi = 150)
    plt.close("all")
    

(1320, 800, 1140)
('AA0282', 'AA0282.swc')
('AA0674', 'AA0674.swc')
('AA0004', 'AA0004.swc')
('AA0471', 'AA0471.swc')
('AA0064', 'AA0064.swc')
('AA0863', 'AA0863.swc')
('AA0652', 'AA0652.swc')
('AA0002', 'AA0002.swc')
('AA0634', 'AA0634.swc')
('AA0801', 'AA0801.swc')
('AA0276', 'AA0276.swc')
('AA0131', 'AA0131.swc')
('AA0475', 'AA0475.swc')
('AA0473', 'AA0473.swc')
('AA0672', 'AA0672.swc')
('AA0666', 'AA0666.swc')
('AA0664', 'AA0664.swc')
('AA0670', 'AA0670.swc')
('AA0603', 'AA0603.swc')
('AA0038', 'AA0038.swc')
('AA0395', 'AA0395.swc')
('AA0836', 'AA0836.swc')
('AA0541', 'AA0541.swc')
('AA0271', 'AA0271.swc')
('AA0799', 'AA0799.swc')
('AA0186', 'AA0186.swc')
('AA0180', 'AA0180.swc')
('AA0628', 'AA0628.swc')
('AA0868', 'AA0868.swc')
('AA0544', 'AA0544.swc')
('AA0181', 'AA0181.swc')
('AA0748', 'AA0748.swc')
('AA0699', 'AA0699.swc')
('AA0856', 'AA0856.swc')
('AA0300', 'AA0300.swc')
('AA0899', 'AA0899.swc')
('AA0639', 'AA0639.swc')
('AA0956', 'AA0956.swc')
('AA0589', 'AA0589.swc')
('AA023

('AA0662', 'AA0662.swc')
('AA0187', 'AA0187.swc')
('AA0005', 'AA0005.swc')
('AA0134', 'AA0134.swc')
('AA0014', 'AA0014.swc')
('AA0291', 'AA0291.swc')
('AA0770', 'AA0770.swc')
('AA0104', 'AA0104.swc')
('AA0440', 'AA0440.swc')
('AA0800', 'AA0800.swc')
('AA1088', 'AA1088.swc')
('AA0642', 'AA0642.swc')
('AA0605', 'AA0605.swc')
('AA0450', 'AA0450.swc')
('AA0798', 'AA0798.swc')
('AA0780', 'AA0780.swc')
('AA0775', 'AA0775.swc')
('AA0445', 'AA0445.swc')
('AA1042', 'AA1042.swc')
('AA0945', 'AA0945.swc')
('AA0626', 'AA0626.swc')
('AA0105', 'AA0105.swc')
('AA0632', 'AA0632.swc')
('AA0286', 'AA0286.swc')
('AA0656', 'AA0656.swc')
('AA0797', 'AA0797.swc')
('AA0773', 'AA0773.swc')
('AA0421', 'AA0421.swc')
('AA0837', 'AA0837.swc')
('AA0617', 'AA0617.swc')
('AA0113', 'AA0113.swc')
('AA0245', 'AA0245.swc')
('AA0741', 'AA0741.swc')
('AA0802', 'AA0802.swc')
('AA0667', 'AA0667.swc')
('AA0577', 'AA0577.swc')
('AA0764', 'AA0764.swc')
('AA0913', 'AA0913.swc')
('AA0596', 'AA0596.swc')
('AA0892', 'AA0892.swc')
